In [1]:
# -*- coding: utf-8 -*-
from hyperparams import Hyperparams as hp
import os
import codecs
from jamo import h2j, j2hcj
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import librosa
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [2]:
trascript = hp.transcript_pos
lines = codecs.open(trascript, 'r', 'utf-8').readlines()

if not (os.path.isdir(hp.mels_dir)):
    os.mkdir(hp.mels_dir)
    print('{%s} does not exist, created {%s}'.format(hp.mels_dir, hp.mels_dir))
    
if not (os.path.isdir(hp.mags_dir)):
    os.mkdir(hp.mags_dir)
    print('{%s} does not exist, created {%s}'.format(hp.mags_dir, hp.mags_dir))

In [3]:
def load_vocab():
    char2idx = {char: idx for idx, char in enumerate(hp.vocab)}
    idx2char = {idx: char for idx, char in enumerate(hp.vocab)}
    return char2idx, idx2char

In [6]:
fnames, texts, secs, text_lengths = [], [], [], []

for line in tqdm(lines):
    line = j2hcj(line)
    _, _, text, _ = line.strip().split('|')
    text_lengths.append(len(text))

maxlen = max(text_lengths)

char2idx, idx2char = load_vocab();

for line in tqdm(lines):
    line = j2hcj(line)
    fname, _, text, sec = line.strip().split('|')
    
    padLen = maxlen - len(text)
    
    text = text + ' ' * padLen
    
    encodedText = [char2idx[char] for char in text]
    
#     encodedText = np.array(encodedText, np.int32).tostring()
    
    fnames.append(fname); 
    texts.append(encodedText)
    secs.append(float(sec)); 

100%|█████████████████████████████████████████████████████████████████████████| 12853/12853 [00:01<00:00, 10376.30it/s]


In [22]:
from keras.layers import Embedding
from keras.initializers import TruncatedNormal
from keras import Input
from keras.models import Model
from keras import layers
from keras.layers import BatchNormalization

In [8]:
from keras import preprocessing

In [76]:
padded_texts = preprocessing.sequence.pad_sequences(texts)

In [9]:
embed_layer = Embedding(len(hp.vocab), 
                        hp.embed_size, 
                        input_length = maxlen,
                        embeddings_initializer=TruncatedNormal(mean=0.0, stddev=0.05, seed=None))

In [10]:
text_input = Input(shape=(None, ), dtype='int32', name='text')

In [11]:
embedded_text = embed_layer(text_input)

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D

In [13]:
prenet_layers = Sequential()
prenet_layers.add(Dense(hp.num_prenet_node_1, activation='relu'))
prenet_layers.add(Dropout(hp.dropout_rate))
prenet_layers.add(Dense(hp.num_prenet_node_2, activation='relu'))
prenet_layers.add(Dropout(hp.dropout_rate))

In [77]:
prenet_layers.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 104, 256)          65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 104, 256)          0         
_________________________________________________________________
dense_2 (Dense)              (None, 104, 128)          32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 104, 128)          0         
Total params: 98,688
Trainable params: 98,688
Non-trainable params: 0
_________________________________________________________________


In [15]:
prenet_result = prenet_layers(embedded_text)

In [16]:
conv1D_outputs = []
for k in range(1, hp.K + 1):
    conv1D_outputs.append(Conv1D(filters=hp.num_k_filter, kernel_size=k, padding='same')(prenet_result))

In [17]:
conv1D_result = layers.concatenate(conv1D_outputs, axis=-1)

In [18]:
max_pooled_result = MaxPooling1D(pool_size=2, strides=1, padding='same')(conv1D_result)

In [19]:
conv_proj_1_result = Conv1D(filters=hp.num_conv1d_proj_filter, 
                            kernel_size=hp.size_conv1d_proj_filter, 
                            padding='same')(max_pooled_result)

In [23]:
norm_1_result = BatchNormalization(axis=1,  
                   epsilon=0.001,
                   center=True, 
                   scale=True)(conv_proj_1_result)

In [24]:
conv_proj_2_result = Conv1D(filters=hp.num_conv1d_proj_filter, 
                            kernel_size=hp.size_conv1d_proj_filter, 
                            padding='same')(norm_1_result)

In [25]:
norm_2_result = BatchNormalization(axis=1,  
                   epsilon=0.001,
                   center=True, 
                   scale=True)(conv_proj_2_result)

In [26]:
residual_result = layers.add([prenet_result, norm_2_result])

In [71]:
'''
https://gist.github.com/iskandr/a874e4cf358697037d14a17020304535
'''

import keras.backend as K
from keras.layers import Dense, Activation, Multiply, Add, Lambda
import keras.initializers

def highway_layers(value, n_layers, activation="tanh", gate_bias=-3):
    dim = K.int_shape(value)[-1]
    gate_bias_initializer = keras.initializers.Constant(gate_bias)
    for i in range(n_layers):     
        gate = Dense(units=dim, bias_initializer=gate_bias_initializer)(value)
        gate = Activation("sigmoid")(gate)
        negated_gate = Lambda(
            lambda x: 1.0 - x,
            output_shape=(dim,))(gate)
        transformed = Dense(units=dim)(value)
        transformed = Activation(activation)(transformed)
        transformed_gated = Multiply()([gate, transformed])
        identity_gated = Multiply()([negated_gate, value])
        value = Add()([transformed_gated, identity_gated])
    return value

In [74]:
highway_result = highway_layers(residual_result, 4)

In [84]:
highway_result

<tf.Tensor 'add_15/add:0' shape=(?, 104, 128) dtype=float32>

In [80]:
from keras.layers import GRU, CuDNNGRU

In [65]:
residual_result

<tf.Tensor 'add_1/add:0' shape=(?, 104, 128) dtype=float32>

In [88]:
gru_result = layers.Bidirectional(CuDNNGRU(128), merge_mode='concat')(highway_result)

In [89]:
gru_result

<tf.Tensor 'bidirectional_4/concat_2:0' shape=(?, 256) dtype=float32>

In [86]:
model = Model(text_input, gru_result)
x = model.predict({'text': padded_texts[0:5]})
print(x.shape)

(5, 256)


In [78]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [117]:
import librosa

def py_func(fname):

    npy_name = fname.split('/')[1].replace('wav', 'npy')
    mel_path = os.path.join(hp.mels_dir, npy_name)
    mag_path = os.path.join(hp.mags_dir, npy_name)
    fpath = os.path.join(hp.data_dir, fname)
    wav, fs = librosa.core.load(fpath, mono=True)

    return fs

In [118]:
name_input = Input(shape=(9, 10))
lll = Lambda(py_func)(name_input)

AttributeError: 'Tensor' object has no attribute 'split'

In [110]:
lll

<tf.Tensor 'lambda_18/Identity:0' shape=(?, 9, 10) dtype=float32>

In [90]:
from keras import backend as K
from keras.layers import Layer

class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [ ]:
with tf.variable_scope("embedding", reuse=tf.AUTO_REUSE):
    lookup_table = tf.get_variable('lookup_table', 
                                   dtype=tf.float32, 
                                   shape=[len(hp.vocab), hp.embed_size],
                                   initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.01))
    

In [ ]:
embed_text = tf.nn.embedding_lookup(lookup_table, text_decoded)
embed_text = tf.expand_dims(embed_text, 0)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(embed_text)
    plt.imshow(x[0, :, :])

In [ ]:
with tf.variable_scope("prenet", reuse=tf.AUTO_REUSE):
    outputs = tf.layers.dense(embed_text, units=hp.num_prenet_node_1, activation=tf.nn.relu, name="dense1")
    outputs = tf.layers.dropout(outputs, rate=hp.dropout_rate, name="dropout1")
    outputs = tf.layers.dense(outputs, units=hp.num_prenet_node_2, activation=tf.nn.relu, name="dense2")
    outputs = tf.layers.dropout(outputs, rate=hp.dropout_rate, name="dropout2") 

    prenet_result = outputs

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(prenet_result)
    plt.imshow(x[0, :, :])

In [ ]:
with tf.variable_scope("conv1d_banks", reuse=tf.AUTO_REUSE):
    for k in range(1, hp.K + 1):
        with tf.variable_scope("filter_num_{}".format(k)):
            params = {"inputs":prenet_result, "filters":hp.num_k_filter, "kernel_size":k,
                    "dilation_rate":1, "padding":"SAME", "activation":None, 
                    "use_bias":False, "reuse":tf.AUTO_REUSE}

            # Works when resue = True
            # For i loop, filter is reused.

            conv_outputs = tf.layers.conv1d(**params)
            if k == 1:
                conv_bank_outputs = conv_outputs
            else:
                conv_bank_outputs = tf.concat((conv_bank_outputs, conv_outputs), axis=2)

    conv_bank_result = conv_bank_outputs

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(conv_bank_result)
    plt.imshow(x[0, :, :], aspect='auto')

In [ ]:
 max_pooling_result = tf.layers.max_pooling1d(conv_bank_result, pool_size=2, strides=1, padding="same")

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(max_pooling_result)
    plt.imshow(x[0, :, :], aspect='auto')

In [ ]:
conv_bank_result.shape

In [ ]:
with tf.variable_scope("conv1d_1"):       
    params = {"inputs":max_pooling_result, "filters":hp.num_conv1d_proj_filter, "kernel_size":hp.size_conv1d_proj_filter,
                    "dilation_rate":1, "padding":"SAME", "activation":None, 
                    "use_bias":False, "reuse":tf.AUTO_REUSE}
    conv_proj_1_result = tf.layers.conv1d(**params)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(conv_proj_1_result)
    plt.imshow(x[0, :, :])

In [ ]:
with tf.variable_scope("conv1d_1"):
    bn_1_result = tf.contrib.layers.batch_norm(inputs=conv_proj_1_result,
                                           center=True,
                                           scale=True,
                                           updates_collections=None,
                                           is_training=True,
                                           scope="conv1d_1",
                                           fused=True,
                                           reuse=tf.AUTO_REUSE)
    batch_norm_1_result = tf.nn.relu(bn_1_result)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(batch_norm_1_result)
    plt.imshow(x[0, :, :])

In [ ]:
with tf.variable_scope("conv1d_2"):
    params = {"inputs":batch_norm_1_result, "filters":hp.num_conv1d_proj_filter, "kernel_size":hp.size_conv1d_proj_filter,
                    "dilation_rate":1, "padding":"SAME", "activation":None, 
                    "use_bias":False, "reuse":tf.AUTO_REUSE}
    conv_proj_2_result = tf.layers.conv1d(**params)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(conv_proj_2_result)
    plt.imshow(x[0, :, :])

In [ ]:
with tf.variable_scope("conv1d_2"):
    bn_2_result = tf.contrib.layers.batch_norm(inputs=conv_proj_2_result,
                                           center=True,
                                           scale=True,
                                           updates_collections=None,
                                           is_training=True,
                                           scope="conv1d_2",
                                           fused=True,
                                           reuse=tf.AUTO_REUSE)
    batch_norm_2_result = tf.nn.relu(bn_2_result)


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(batch_norm_2_result)
    plt.imshow(x[0, :, :])

In [ ]:
res_output = prenet_result + batch_norm_2_result

In [ ]:
res_output

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(res_output)
    plt.imshow(x[0, :, :])

In [ ]:
highwaynet_output = []

for i in range(hp.num_highwaynet_blocks):
    scope = "highwaynet_{:d}".format(i)
    with tf.variable_scope(scope):
        
        if i == 0:
            highwaynet_input = res_output
        else:
            highwaynet_input = highwaynet_output

        H = tf.layers.dense(highwaynet_input, units=hp.num_highwaynet_units, activation=tf.nn.relu, name="dense1", reuse=tf.AUTO_REUSE)
        T = tf.layers.dense(highwaynet_input, units=hp.num_highwaynet_units, activation=tf.nn.sigmoid,
                            bias_initializer=tf.constant_initializer(-1.0), name="dense2", reuse=tf.AUTO_REUSE)
        highwaynet_output = H*T + highwaynet_input*(1.-T)    

In [ ]:
highwaynet_output

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(highwaynet_output)
    plt.imshow(x[0, :, :])

In [ ]:
with tf.variable_scope("gru", reuse=tf.AUTO_REUSE):
#     cell = tf.contrib.rnn.GRUCell(hp.num_gru_units)
#     cell_bw = tf.contrib.rnn.GRUCell(hp.num_gru_units)
    cell = tf.contrib.rnn.GRUCell(128)
    cell_bw = tf.contrib.rnn.GRUCell(128)

    output, _ = tf.nn.bidirectional_dynamic_rnn(cell, cell_bw, highwaynet_output, dtype=tf.float32)
    gru_result = tf.concat(output, 2)   

In [ ]:
gru_result

In [ ]:
decoder_input

In [ ]:
with tf.variable_scope("decoder_prenet", reuse=tf.AUTO_REUSE):
    outputs = tf.layers.dense(decoder_input, units=hp.num_prenet_node_1, activation=tf.nn.relu, name="dense1")
    outputs = tf.layers.dropout(outputs, rate=hp.dropout_rate, name="dropout1")
    outputs = tf.layers.dense(outputs, units=hp.num_prenet_node_2, activation=tf.nn.relu, name="dense2")
    decoder_prenet_result = tf.layers.dropout(outputs, rate=hp.dropout_rate, name="dropout2") 

In [ ]:
decoder_prenet_result

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(decoder_prenet_result)
    plt.imshow(x)

In [ ]:
decoder_prenet_result

In [ ]:
gru_result

In [ ]:
decoder_prenet_result_4D

In [ ]:
with tf.variable_scope("attention_decoder", reuse=tf.AUTO_REUSE):
    
    decoder_prenet_result_4D = tf.expand_dims(decoder_prenet_result, 0)

    attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(hp.num_attention_units, gru_result)
    decoder_cell = tf.contrib.rnn.GRUCell(hp.num_gru_units)
    cell_with_attention = tf.contrib.seq2seq.AttentionWrapper(decoder_cell,
                                                              attention_mechanism,
                                                              hp.num_attention_units,
                                                              alignment_history=True)
    dec, state = tf.nn.dynamic_rnn(cell_with_attention, decoder_prenet_result_4D, dtype=tf.float32)
#     dec, state = keras.layers.RNN(cell_with_attention, decoder_prenet_result_4D, dtype=tf.float32)

In [ ]:
decoder_prenet_result_4D

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(state)
    x

In [ ]:
x.shape

In [ ]:
with tf.variable_scope("attention_decoder", reuse=tf.AUTO_REUSE):
    
    alignment = tf.transpose(state.alignment_history.stack(),[1,2,0])

    with tf.variable_scope("decoder_gru_1", reuse=tf.AUTO_REUSE):
        cell = tf.contrib.rnn.GRUCell(128)
        output, _ = tf.nn.dynamic_rnn(cell, dec, dtype=tf.float32)
        gru_output_1 = tf.concat(output, 2)

    dec = dec + gru_output_1

    with tf.variable_scope("decoder_gru_2", reuse=tf.AUTO_REUSE):
        cell = tf.contrib.rnn.GRUCell(128)
        output, _ = tf.nn.dynamic_rnn(cell, dec, dtype=tf.float32)
        gru_output_2 = tf.concat(output, 2)

    dec = dec + gru_output_2

    # Outputs => (N, T_y/r, n_mels*r)
    y_hat = tf.layers.dense(dec, hp.n_mels*hp.r)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(y_hat)
    plt.imshow(x[0, :, :])

In [ ]:
with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
    dec_2_input = tf.reshape(y_hat, [1, -1, hp.n_mels])

In [ ]:
with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
    with tf.variable_scope("conv1d_banks", reuse=tf.AUTO_REUSE):
        for k in range(1, hp.K + 1):
            with tf.variable_scope("filter_num_{}".format(k)):
                params = {"inputs":dec_2_input, "filters":hp.num_k_filter, "kernel_size":k,
                        "dilation_rate":1, "padding":"SAME", "activation":None, 
                        "use_bias":False, "reuse":tf.AUTO_REUSE}

                # Works when resue = True
                # For i loop, filter is reused.

                conv_outputs = tf.layers.conv1d(**params)
                if k == 1:
                    conv_bank_outputs = conv_outputs
                else:
                    conv_bank_outputs = tf.concat((conv_bank_outputs, conv_outputs), axis=2)

    dec_2_conv_bank_result = conv_bank_outputs


In [ ]:
with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
    dec_2_max_pooling_result = tf.layers.max_pooling1d(dec_2_conv_bank_result, pool_size=2, strides=1, padding="same")

In [ ]:
with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
    with tf.variable_scope("conv1d_1"):
        params = {"inputs":dec_2_max_pooling_result, "filters":hp.num_conv1d_proj_filter, "kernel_size":hp.size_conv1d_proj_filter,
                        "dilation_rate":1, "padding":"SAME", "activation":None, 
                        "use_bias":False, "reuse":tf.AUTO_REUSE}

        dec_2_conv_proj_1_result = tf.layers.conv1d(**params)

In [ ]:
with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
    with tf.variable_scope("conv1d_1"):
        bn_1_result = tf.contrib.layers.batch_norm(inputs=dec_2_conv_proj_1_result,
                                               center=True,
                                               scale=True,
                                               updates_collections=None,
                                               is_training=True,
                                               scope="conv1d_1",
                                               fused=True,
                                               reuse=tf.AUTO_REUSE)
        dec_2_batch_norm_1_result = tf.nn.relu(bn_1_result)

In [ ]:
with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
    with tf.variable_scope("conv1d_2"):
        params = {"inputs":dec_2_batch_norm_1_result, "filters":hp.num_conv1d_proj_filter, "kernel_size":hp.size_conv1d_proj_filter,
                        "dilation_rate":1, "padding":"SAME", "activation":None, 
                        "use_bias":False, "reuse":tf.AUTO_REUSE}

        dec_2_conv_proj_2_result = tf.layers.conv1d(**params)

In [ ]:
with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
    with tf.variable_scope("conv1d_2"):
        bn_2_result = tf.contrib.layers.batch_norm(inputs=dec_2_conv_proj_2_result,
                                               center=True,
                                               scale=True,
                                               updates_collections=None,
                                               is_training=True,
                                               scope="conv1d_2",
                                               fused=True,
                                               reuse=tf.AUTO_REUSE)
        dec_2_batch_norm_2_result = tf.nn.relu(bn_2_result)

In [ ]:
with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
    dec_2_sync_result = tf.layers.dense(dec_2_batch_norm_2_result, 128)

In [ ]:
with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
    for i in range(hp.num_highwaynet_blocks):
        scope = "highwaynet_{:d}".format(i)
        with tf.variable_scope(scope):

            if i == 0:
                highwaynet_input = dec_2_sync_result
            else:
                highwaynet_input = highwaynet_output

            highwaynet_output = []

            H = tf.layers.dense(highwaynet_input, units=hp.num_highwaynet_units, activation=tf.nn.relu, name="dense1", reuse=tf.AUTO_REUSE)
            T = tf.layers.dense(highwaynet_input, units=hp.num_highwaynet_units, activation=tf.nn.sigmoid,
                                bias_initializer=tf.constant_initializer(-1.0), name="dense2", reuse=tf.AUTO_REUSE)
            highwaynet_output = H*T + highwaynet_input*(1.-T)

In [ ]:
with tf.variable_scope("dec_2_gru", reuse=tf.AUTO_REUSE):
    cell = tf.contrib.rnn.GRUCell(128)
    cell_bw = tf.contrib.rnn.GRUCell(128)

    output, _ = tf.nn.bidirectional_dynamic_rnn(cell, cell_bw, highwaynet_output, dtype=tf.float32)
    dec_2_gru_result = tf.concat(output, 2)


In [ ]:
with tf.variable_scope("decoder2", reuse=tf.AUTO_REUSE):
    with tf.variable_scope("final", reuse=tf.AUTO_REUSE):
        z_hat = tf.layers.dense(dec_2_gru_result, 1 + hp.nsc_sec*hp.fs//2)

In [ ]:
z_hat

In [ ]:
mag[0].shape

In [ ]:
loss1 = tf.reduce_mean(tf.abs(y_hat - y))
loss2 = tf.reduce_mean(tf.abs(z_hat - mag))
loss = loss1 + loss2

In [ ]:
loss

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(loss)
    print(x)

In [ ]:
global_step = tf.Variable(0, name='global_step', trainable=False)
lr = learning_rate_decay(hp.lr, global_step=global_step)
optimizer = tf.train.AdamOptimizer(learning_rate=hp.lr)

## gradient clipping
gvs = optimizer.compute_gradients(loss)

In [ ]:
clipped = []

for grad, var in gvs:
    if grad is None:
        clipped.append((grad, var))
    else :
        grad = tf.clip_by_norm(grad, 5.)
        clipped.append((grad, var))
    
train_op = optimizer.apply_gradients(clipped, global_step=global_step)

In [ ]:
mode = 'train'

tf.summary.scalar('{}/loss1'.format(mode), loss1)
tf.summary.scalar('{}/loss'.format(mode), loss)
tf.summary.scalar('{}/lr'.format(mode), hp.lr)

tf.summary.image("{}/mel_gt".format(mode), tf.expand_dims(y, -1), max_outputs=1)
tf.summary.image("{}/mel_hat".format(mode), tf.expand_dims(y_hat, -1), max_outputs=1)
tf.summary.image("{}/mag_gt".format(mode), tf.expand_dims(mag, -1), max_outputs=1)
tf.summary.image("{}/mag_hat".format(mode), tf.expand_dims(z_hat, -1), max_outputs=1)

merged = tf.summary.merge_all()

In [ ]:
with tf.Session() as sess:
    for i in range(100):
        sess.run(tf.global_variables_initializer())
        _, gs = sess.run([train_op, global_step])
        print(gs)

In [ ]:
hp.logdir = '.'

In [ ]:
with tf.train.MonitoredTrainingSession(summary_dir=hp.logdir, save_summaries_secs=60) as sess:
    while 1:
        for _ in tqdm(range(100), total=100, ncols=70, leave=False, unit='b'):
            #_, gs = sess.run([train_op, global_step])
            
            _, gs = sess.run([dec_2_sync_result])

            # Write checkpoint files
            if gs % 1000 == 0:
                sv.saver.save(sess, hp.logdir + '/model_gs_{}k'.format(gs//1000))

                # plot the first alignment for logging
                al = sess.run(alignment)
                plt.imshow(al[0])
                ply.show()